In [ ]:
! pip install sweetviz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 34.7 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from google.colab import drive 
#import sweetviz as sv
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from lightgbm import LGBMClassifier
import xgboost as xgb
from sklearn.ensemble import VotingClassifier, StackingClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score, roc_auc_score, confusion_matrix
from imblearn.over_sampling import RandomOverSampler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/9417 GROUP PROJECT/churn_data.csv')

In [ ]:
 from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df.head()

,CustomerID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Gender,...,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Label,Churn Value,Churn Score,CLTV,Churn Reason
0,3668-QPYBK,1,United States,California,Los Angeles,90003,"33.964131, -118.272783",33.964131,-118.272783,Male,...,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,1,86,3239,Competitor made better offer
1,9237-HQITU,1,United States,California,Los Angeles,90005,"34.059281, -118.30742",34.059281,-118.307420,Female,...,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,1,67,2701,Moved
2,9305-CDSKC,1,United States,California,Los Angeles,90006,"34.048013, -118.293953",34.048013,-118.293953,Female,...,Month-to-month,Yes,Electronic check,99.65,820.5,Yes,1,86,5372,Moved
3,7892-POOKP,1,United States,California,Los Angeles,90010,"34.062125, -118.315709",34.062125,-118.315709,Female,...,Month-to-month,Yes,Electronic check,104.80,3046.05,Yes,1,84,5003,Moved
4,0280-XJGEX,1,United States,California,Los Angeles,90015,"34.039224, -118.266293",34.039224,-118.266293,Male,...,Month-to-month,Yes,Bank transfer (automatic),103.70,5036.3,Yes,1,89,5340,Competitor had better devices


In [ ]:
#df = df.drop(['CustomerID', 'Churn Label', 'Churn Score', 'CLTV', 'Churn Reason'], axis=1)
df.isnull().sum()

CustomerID              0
Count                   0
Country                 0
State                   0
City                    0
Zip Code                0
Lat Long                0
Latitude                0
Longitude               0
Gender                  0
Senior Citizen          0
Partner                 0
Dependents              0
Tenure Months           0
Phone Service           0
Multiple Lines          0
Internet Service        0
Online Security         0
Online Backup           0
Device Protection       0
Tech Support            0
Streaming TV            0
Streaming Movies        0
Contract                0
Paperless Billing       0
Payment Method          0
Monthly Charges         0
Total Charges           0
Churn Label             0
Churn Value             0
Churn Score             0
CLTV                    0
Churn Reason         5174
dtype: int64

In [ ]:
result = sv.analyze(df, "Churn Value", pairwise_analysis="on")
result.show_html()

NameError: ignored

In [ ]:

# converts total charges from string to float
df['Total Charges'] = df['Total Charges'].replace(r'^\s*$', np.nan, regex=True)
df['Total Charges'] = df['Total Charges'].astype(float)
#choosing variables apart from latitude, longitude, zip code
df1 = df[['Country', 'State', 'City', 'Gender', 'Senior Citizen', 'Partner', 'Dependents', 'Tenure Months', 'Phone Service', 'Multiple Lines', 'Internet Service', 'Online Security','Online Backup', 'Device Protection', 'Tech Support', 'Streaming TV','Streaming Movies', 'Contract', 'Paperless Billing', 'Payment Method','Monthly Charges', 'Total Charges', 'CLTV','Churn Value']]
df1.head()


,Country,State,City,Gender,Senior Citizen,Partner,Dependents,Tenure Months,Phone Service,Multiple Lines,...,Tech Support,Streaming TV,Streaming Movies,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,CLTV,Churn Value
0,United States,California,Los Angeles,Male,No,No,No,2,Yes,No,...,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,3239,1
1,United States,California,Los Angeles,Female,No,No,Yes,2,Yes,No,...,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,2701,1
2,United States,California,Los Angeles,Female,No,No,Yes,8,Yes,Yes,...,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.50,5372,1
3,United States,California,Los Angeles,Female,No,Yes,Yes,28,Yes,Yes,...,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,104.80,3046.05,5003,1
4,United States,California,Los Angeles,Male,No,No,Yes,49,Yes,Yes,...,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),103.70,5036.30,5340,1


In [ ]:
#one hot encodeing all variables
df_encoded = pd.get_dummies(df1, columns=['Gender', 'Senior Citizen', 'Partner', 'Dependents','Phone Service', 'Multiple Lines', 'Internet Service', 'Online Security','Online Backup', 'Device Protection', 'Tech Support', 'Streaming TV','Streaming Movies', 'Contract', 'Paperless Billing', 'Payment Method'])

In [ ]:
# Dropping dummy variables
#df_encoded = df_encoded.drop(['Gender_Female', 'Senior Citizen_No', 'Partner_No', 'Paperless Billing_No', 'Dependents_No', 'Phone Service_No', 'Multiple Lines_No', 'Internet Service_No', 'Online Security_No', 'Online Backup_No', 'Device Protection_No', 'Tech Support_No', 'Streaming TV_No', 'Streaming Movies_No', 'Contract_One year', 'Payment Method_Mailed check'], axis = 1)
df_encoded = df_encoded.drop(['Multiple Lines_No phone service', 'Online Security_No internet service', 'Online Backup_No internet service','Device Protection_No internet service', 'Tech Support_No internet service', 'Streaming TV_No internet service', 'Streaming Movies_No internet service', 'Gender_Female', 'Senior Citizen_No', 'Partner_No', 'Paperless Billing_No', 'Dependents_No', 'Phone Service_No', 'Multiple Lines_No', 'Internet Service_No', 'Online Security_No', 'Online Backup_No', 'Device Protection_No', 'Tech Support_No', 'Streaming TV_No', 'Streaming Movies_No', 'Contract_One year', 'Payment Method_Mailed check'], axis = 1)
# Scaling continuous variable
scaler = StandardScaler()
df_encoded[['Tenure Months', 'Monthly Charges', 'Total Charges', 'CLTV']] = scaler.fit_transform(df_encoded[['Tenure Months', 'Monthly Charges', 'Total Charges', 'CLTV']])
df_encoded.head()

,Country,State,City,Tenure Months,Monthly Charges,Total Charges,CLTV,Churn Value,Gender_Male,Senior Citizen_Yes,...,Device Protection_Yes,Tech Support_Yes,Streaming TV_Yes,Streaming Movies_Yes,Contract_Month-to-month,Contract_Two year,Paperless Billing_Yes,Payment Method_Bank transfer (automatic),Payment Method_Credit card (automatic),Payment Method_Electronic check
0,United States,California,Los Angeles,-1.236724,-0.362660,-0.959649,-0.981675,1,1,0,...,0,0,0,0,1,0,1,0,0,0
1,United States,California,Los Angeles,-1.236724,0.197365,-0.940457,-1.436462,1,0,0,...,0,0,0,0,1,0,1,0,0,1
2,United States,California,Los Angeles,-0.992402,1.159546,-0.645369,0.821409,1,0,0,...,1,0,1,1,1,0,1,0,0,1
3,United States,California,Los Angeles,-0.177995,1.330711,0.336516,0.509483,1,0,0,...,1,1,1,1,1,0,1,0,0,1
4,United States,California,Los Angeles,0.677133,1.294151,1.214589,0.794358,1,1,0,...,1,0,1,1,1,0,1,1,0,0


## Using SVM for Feature Selection

In [ ]:
from sklearn.datasets import make_classification
from sklearn.feature_selection import RFE
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn import svm
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVR

In [ ]:

df_encoded = df_encoded.dropna()
df_encoded

,Country,State,City,Tenure Months,Monthly Charges,Total Charges,CLTV,Churn Value,Gender_Male,Senior Citizen_Yes,...,Device Protection_Yes,Tech Support_Yes,Streaming TV_Yes,Streaming Movies_Yes,Contract_Month-to-month,Contract_Two year,Paperless Billing_Yes,Payment Method_Bank transfer (automatic),Payment Method_Credit card (automatic),Payment Method_Electronic check
0,United States,California,Los Angeles,-1.236724,-0.362660,-0.959649,-0.981675,1,1,0,...,0,0,0,0,1,0,1,0,0,0
1,United States,California,Los Angeles,-1.236724,0.197365,-0.940457,-1.436462,1,0,0,...,0,0,0,0,1,0,1,0,0,1
2,United States,California,Los Angeles,-0.992402,1.159546,-0.645369,0.821409,1,0,0,...,1,0,1,1,1,0,1,0,0,1
3,United States,California,Los Angeles,-0.177995,1.330711,0.336516,0.509483,1,0,0,...,1,1,1,1,1,0,1,0,0,1
4,United States,California,Los Angeles,0.677133,1.294151,1.214589,0.794358,1,1,0,...,1,0,1,1,1,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,United States,California,Landers,1.613701,-1.449476,-0.381142,0.765617,0,0,0,...,0,0,0,0,0,1,1,1,0,0
7039,United States,California,Adelanto,-0.340876,0.665992,-0.129180,-1.910691,0,1,0,...,1,1,1,1,0,0,1,0,0,0
7040,United States,California,Amboy,1.613701,1.277533,2.241056,0.980330,0,0,0,...,1,0,1,1,0,0,1,0,1,0
7041,United States,California,Angelus Oaks,-0.870241,-1.168632,-0.854514,-1.358692,0,0,0,...,0,0,0,0,1,0,1,0,0,1


In [ ]:
y = df_encoded['Churn Value']
y

0       1
1       1
2       1
3       1
4       1
       ..
7038    0
7039    0
7040    0
7041    0
7042    0
Name: Churn Value, Length: 7032, dtype: int64

In [ ]:
X = df_encoded.drop(['Churn Value', 'Country','State','City'], axis = 1)
X

,Tenure Months,Monthly Charges,Total Charges,CLTV,Gender_Male,Senior Citizen_Yes,Partner_Yes,Dependents_Yes,Phone Service_Yes,Multiple Lines_Yes,...,Device Protection_Yes,Tech Support_Yes,Streaming TV_Yes,Streaming Movies_Yes,Contract_Month-to-month,Contract_Two year,Paperless Billing_Yes,Payment Method_Bank transfer (automatic),Payment Method_Credit card (automatic),Payment Method_Electronic check
0,-1.236724,-0.362660,-0.959649,-0.981675,1,0,0,0,1,0,...,0,0,0,0,1,0,1,0,0,0
1,-1.236724,0.197365,-0.940457,-1.436462,0,0,0,1,1,0,...,0,0,0,0,1,0,1,0,0,1
2,-0.992402,1.159546,-0.645369,0.821409,0,0,0,1,1,1,...,1,0,1,1,1,0,1,0,0,1
3,-0.177995,1.330711,0.336516,0.509483,0,0,1,1,1,1,...,1,1,1,1,1,0,1,0,0,1
4,0.677133,1.294151,1.214589,0.794358,1,0,0,1,1,1,...,1,0,1,1,1,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,1.613701,-1.449476,-0.381142,0.765617,0,0,0,0,1,0,...,0,0,0,0,0,1,1,1,0,0
7039,-0.340876,0.665992,-0.129180,-1.910691,1,0,1,1,1,1,...,1,1,1,1,0,0,1,0,0,0
7040,1.613701,1.277533,2.241056,0.980330,0,0,1,1,1,1,...,1,0,1,1,0,0,1,0,1,0
7041,-0.870241,-1.168632,-0.854514,-1.358692,0,0,1,1,0,0,...,0,0,0,0,1,0,1,0,0,1


In [ ]:
svc = LinearSVC()
svc.fit(X, y)

# Get the absolute values of the coefficients
coef = svc.coef_[0]
abs_coef = [abs(c) for c in coef]

# Print the feature ranking
print("Feature ranking:")
for i in range(X.shape[1]):
    print(f"Feature {i}: {abs_coef[i]}")

Feature ranking:
Feature 0: 0.3488195766488868
Feature 1: 0.06560144585399348
Feature 2: 0.021556248878635463
Feature 3: 0.009239013930036403
Feature 4: 0.012122389653536424
Feature 5: 0.0463260051213816
Feature 6: 0.10164591446706034
Feature 7: 0.5012826212150202
Feature 8: 0.2289166028101403
Feature 9: 0.10482763008904626
Feature 10: 0.2043159590643383
Feature 11: 0.4671857627480429
Feature 12: 0.13310400625614774
Feature 13: 0.06001115072313125
Feature 14: 0.0311460780636948
Feature 15: 0.13968682511156721
Feature 16: 0.06489706271118578
Feature 17: 0.0693008183259125
Feature 18: 0.21569339907878948
Feature 19: 0.026900655008716603
Feature 20: 0.12060899415769474
Feature 21: 0.01697664274536951
Feature 22: 0.0015005686880551628
Feature 23: 0.14686858846707745


In [ ]:
svc = SVC(kernel="linear", C=1)
svc.fit(X, y)

# Use SelectFromModel to select the most important features
model = SelectFromModel(svc, prefit=True)
X_new = model.transform(X)

print("Original number of features:", X.shape[1])
print("Reduced number of features:", X_new.shape[1])

Original number of features: 24
Reduced number of features: 7


/usr/local/lib/python3.9/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


In [ ]:
# Get the Boolean mask indicating which features have been selected
selected_features_mask = model.get_support()

# Get the names of the selected features
selected_features = X.columns[selected_features_mask]

print("Selected features:", selected_features)

Selected features: Index(['Tenure Months', 'Dependents_Yes', 'Phone Service_Yes',
       'Internet Service_DSL', 'Internet Service_Fiber optic',
       'Tech Support_Yes', 'Contract_Month-to-month'],
      dtype='object')


In [ ]:
# Create an SVM model with a linear kernel
svm = SVC(kernel='linear')

# Create an RFE object with the SVM model and a specified number of top features to select
# You can change the 'n_features_to_select' parameter according to your needs
rfe = RFE(estimator=svm, n_features_to_select=10, step=1)

# Fit the RFE object to your data
rfe.fit(X, y)

# Get the boolean mask of the selected features
selected_features_mask = rfe.get_support()

# Get the column names of the selected features
selected_features = X.columns[selected_features_mask]

print("Selected Features:", selected_features)

Selected Features: Index(['Tenure Months', 'Monthly Charges', 'Dependents_Yes',
       'Multiple Lines_Yes', 'Internet Service_DSL',
       'Internet Service_Fiber optic', 'Streaming TV_Yes',
       'Streaming Movies_Yes', 'Contract_Month-to-month',
       'Payment Method_Electronic check'],
      dtype='object')


Using the above two SVM feature selection methods, and other information that Monthly Charges has high correlation with other variables we decide the following as our selected features. 

We also need to include 'Contact_Two year' as well since Contract is a dummy variable with three possible options, requiring the use of two dummy variables in any model.

We remove payment as none of the 4 other payment dummy variables were included in RFE and the SVM didnt have any payment dummies either saying that combined they have little influence.

In [ ]:
X = X[['Tenure Months', 'Dependents_Yes', 'Phone Service_Yes',
       'Multiple Lines_Yes', 'Internet Service_DSL',
       'Internet Service_Fiber optic', 'Tech Support_Yes', 'Streaming TV_Yes',
       'Contract_Month-to-month', 'Contract_Two year']]

The RFE method, which is the widely accepted method for svm feature selection gives mostly same features as our original svm method for feature selection, hence allowing us to combine the two feature selection methods to create one final version. 
